# Automatically add article/journal info to a Google Scholar profile
Export your Google Scholar profile as CSV,

upload the file(s) from [here](./),

This program will add new columns like:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Institution Authors</th>
      <th>Institution Groups</th>
      <th>DOI</th>
      <th>ISSN</th>
      <th>Impact Factor</th>
      <th>Quartil</th>
      <th>Journal Hindex</th>
      <th>...</th>
    </tr>
  </thead>
</table>
The enriched CSV file can be saved and used as a [Spreadsheet](https://docs.google.com/spreadsheets/d/19dx09hBthKknZs1XxIKuGgwq3ieyPGx8Xr3hi4xqvBc/edit?usp=sharing).

Full Code in this public [repository](https://github.com/restrepo/articles).

To run the program use `<Shift>+<Enter>` in each Code Cell (or from the Menu: `Cell -> Run Cells`)

In [2]:
from publications import *

Here we assume that there exist a data base with the authors multiple names associated to the real name in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Author_Names</th>
      <th>Control</th>
      <th>Full_Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Zapata, Oscar;Zapata, O.</td>
      <td> 0</td>
      <td> Óscar Alberto Zapata Noreña</td>
    </tr>
  </tbody>
</table>
Where control is some identification number associated to the author.


We assume also that there is a data base with the Group at which some real name author belongs in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Full_Name</th>
      <th>Institution_Group</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Óscar Alberto Zapata Noreña</td>
      <td> GFIF</td>
    </tr>
  </tbody>
</table>

In [22]:
a=articles('citations.csv',user='-6mndWkAAAAJ',authors_file='authors.csv',group_file='groups.csv')

In [23]:
fulldoi=a.articles_update()

Updating entry:0.1.2.

In [24]:
a.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Cites,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,83,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,40,4.643,Q1,259
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics,33,5.81,Q3,70


In [25]:
fulldoi.ix[2]

DOI                                                                                                                                                                                           10.1088/1475-7516/2010/10/033
ISSN                                                                                                                                                                                                            [1475-7516]
URL                                                                                                                                                                         http://dx.doi.org/10.1088/1475-7516/2010/10/033
article-number                                                                                                                                                                                                             
author              [{'affiliation': [], 'family': 'Choi', 'given': 'Ki-Young'}, {'affiliation': [], 'family': 'Restrepo

## Update authors/Grupos data bases

In [26]:
a.add_institution_author()

Full name: Diego Alejandro Restrepo Quintero
Author names
(Example: Perez, Juan;Perez;Pérez, J.):
Restrepo, D;Restrepo, Diego
Additional identitication number: 0


In [27]:
a.add_institution_group()

Find author by first name and first surname
(Example: Juan Pérez);
Diego Restrepo
Group name for
Diego Alejandro Restrepo Quintero
(Example: IA: Inteligencia Artificial)GFIF


Save updated data bases:

In [33]:
a.institution_authors.to_csv('authors.csv',index=False)
a.institution_group.to_csv('groups.csv',index=False)

## Save uptated data base

In [34]:
a.to_csv('newcitations.csv')

Restore saved data base

In [3]:
arts=articles('newcitations.csv',user='-6mndWkAAAAJ',authors_file='authors.csv',group_file='groups.csv')

In [36]:
arts.institution_authors

,Author_Names,Control,Full_Name
0,"Zapata, Oscar;Zapata, O.",0,Óscar Alberto Zapata Noreña
1,"Restrepo, D;Restrepo, Diego",0,Diego Alejandro Restrepo Quintero


In [37]:
arts.institution_group

,Full_Name,Institution_Group
0,Óscar Alberto Zapata Noreña,GFIF
1,Diego Alejandro Restrepo Quintero,GFIF


In [38]:
arts.articles_update()

Updating entry:0.1.2.

""


In [4]:
arts.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Cites,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,83,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,40,4.643,Q1,259
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics,33,5.810,Q3,70


In [48]:
wos=pd.read_excel('wos_all.xlsx').fillna('')

In [49]:
arts.articles.DOI.str.split('http://dx.doi.org/')[1][1].lower()

'10.1103/physrevd.85.023523'

In [50]:
arts.articles.DOI.str.split('http://dx.doi.org/')[0][1].lower()

'10.1103/physrevd.79.013011'

In [51]:
wos[wos.DI.str.lower().str.contains( arts.articles.DOI.str.split('http://dx.doi.org/')[1][1].lower()  )]#arts.articles.DOI.str.split('http://dx.doi.org/')[2][1].lower()]

,AB,AF,AR,AU,BP,DE,DT,EM,EP,FU,FX,GA,ID,IS,JI,LA,NR,PA,PD,PG,PI,PT,PU,PY,RP,SC,SN,SO,TC,TI,UT,VL,WC,DI,PM,RI,EI,OI,CL,CT,CY,SP,SU,BE,BN,HO,PN,SE,MA,SI,GP,CA,BA,BF
4102,Bilinear R-parity violation provides an attractive origin for neutrino masses and mixings. In such schemes the gravitino is a viable decaying dark matter particle whose R-parity violating decays l...,"Restrepo, Diego\nTaoso, Marco\nValle, J. W. F.\nZapata, Oscar\n",023523\n,"Restrepo, D\nTaoso, M\nValle, JWF\nZapata, O\n",,,Article\n,restrepo@udea.edu.co; taoso@ific.uv.es; valle@ific.uv.es;\npfozapata@eia.edu.co\n,,"Spanish MICINN [FPA2008-00319/FPA, CSD2009-00064]; EU [UNILHC\nPITN-GA-2009-237920]; EIA [CI-2009-2]; AHEP group; IFIC;\nSostenibilidad-UdeA/2009 [IN10140-CE]; [Prometeo/2009/091]\n","We acknowledge German Gomez-Vargas for useful discussions. This work was\nsupported by the Spanish MICINN under Grant No. FPA2008-00319/FPA and\nMULTIDARK Consolider CSD2009-00064, by Prometeo/200...",887MU\n,MISSING TRANSVERSE-MOMENTUM; PROTON-PROTON COLLISIONS; SUPERSYMMETRY;\nBREAKING; OSCILLATIONS; CONSTRAINTS; PARTICLE; SEARCH; CMSSM; MODEL\n,2\n,Phys. Rev. D\n,English\n,68,"ONE PHYSICS ELLIPSE, COLLEGE PK, MD 20740-3844 USA\n",JAN 23\n,7,COLLEGE PK\n,J\n,AMER PHYSICAL SOC\n,2012,"Restrepo, D (reprint author), Univ Antioquia, Inst Fis, Medellin 1226, Colombia.\n",Astronomy & Astrophysics; Physics\n,2470-0010\n,PHYSICAL REVIEW D\n,24,Gravitino dark matter and neutrino masses with bilinear R-parity\nviolation\n,WOS:000299932700003\n,85\n,"Astronomy & Astrophysics; Physics, Particles & Fields\n",10.1103/PhysRevD.85.023523\n,,"Restrepo, Diego/E-6977-2013; Valle, J/F-7573-2013\n",2470-0029\n,"Restrepo, Diego/0000-0001-6455-5564; Valle, J/0000-0002-1881-5094\n",,,,,,,,,,,,,,,,


In [39]:
wos[wos.DI.str.lower().str.contains( '10.1103/physrevd.85.023523'.lower()  )]

,AB,AF,AR,AU,BP,DE,DT,EM,EP,FU,FX,GA,ID,IS,JI,LA,NR,PA,PD,PG,PI,PT,PU,PY,RP,SC,SN,SO,TC,TI,UT,VL,WC,DI,PM,RI,EI,OI,CL,CT,CY,SP,SU,BE,BN,HO,PN,SE,MA,SI,GP,CA,BA,BF
4102,Bilinear R-parity violation provides an attractive origin for neutrino masses and mixings. In such schemes the gravitino is a viable decaying dark matter particle whose R-parity violating decays l...,"Restrepo, Diego\nTaoso, Marco\nValle, J. W. F.\nZapata, Oscar\n",023523\n,"Restrepo, D\nTaoso, M\nValle, JWF\nZapata, O\n",774\n,Clementine juice; organoleptic quality; ascorbic acid; flavonoids;\nnutraceutical; volatile profile\n,Article\n,restrepo@udea.edu.co; taoso@ific.uv.es; valle@ific.uv.es;\npfozapata@eia.edu.co\n,781\n,"Spanish MICINN [FPA2008-00319/FPA, CSD2009-00064]; EU [UNILHC\nPITN-GA-2009-237920]; EIA [CI-2009-2]; AHEP group; IFIC;\nSostenibilidad-UdeA/2009 [IN10140-CE]; [Prometeo/2009/091]\n","We acknowledge German Gomez-Vargas for useful discussions. This work was\nsupported by the Spanish MICINN under Grant No. FPA2008-00319/FPA and\nMULTIDARK Consolider CSD2009-00064, by Prometeo/200...",887MU\n,MISSING TRANSVERSE-MOMENTUM; PROTON-PROTON COLLISIONS; SUPERSYMMETRY;\nBREAKING; OSCILLATIONS; CONSTRAINTS; PARTICLE; SEARCH; CMSSM; MODEL\n,2\n,Phys. Rev. D\n,English\n,68,"ONE PHYSICS ELLIPSE, COLLEGE PK, MD 20740-3844 USA\n",JAN 23\n,7,COLLEGE PK\n,J\n,AMER PHYSICAL SOC\n,2012,"Restrepo, D (reprint author), Univ Antioquia, Inst Fis, Medellin 1226, Colombia.\n",Astronomy & Astrophysics; Physics\n,2470-0010\n,PHYSICAL REVIEW D\n,24,Gravitino dark matter and neutrino masses with bilinear R-parity\nviolation\n,WOS:000299932700003\n,85\n,"Astronomy & Astrophysics; Physics, Particles & Fields\n",10.1103/PhysRevD.85.023523\n,2.22254e+07,"Restrepo, Diego/E-6977-2013; Valle, J/F-7573-2013\n",2470-0029\n,"Restrepo, Diego/0000-0001-6455-5564; Valle, J/0000-0002-1881-5094\n","Madrid, SPAIN\n[Vargas, F.] Univ Antioquia, Rheumatol Sect, Medellin, Colombia.\n[Ramirez, L. A.] Rheumatologist Reumalab & Reumatologya SA, Medellin, Colombia.\n",2nd Systemic Sclerosis World Congress\n,"FEB 02-04, 2012\n",Int Soc Bipolar Disorders\n,2\n,"Mendez, JF\nRodriguez, TYA\nVazquez, HV\nMarin, CFC\nBarreda, MC\nRojas, RR\nMedina, JC\nRuiz, SBL\n",978-3-642-21197-3\n,Los Andes Univ\n,3\n,IFMBE Proceedings\n,375-ENVR\n,SI\n,IEEE\n,LACE Consortium\n,"Montegranario, H\nEspinosa, J\n","Montegranario, H\nEspinosa, J\n"


In [18]:
wos[wos.AU.str.contains('Zapata, O')]

,AB,AF,AR,AU,BP,DE,DT,EM,EP,FU,FX,GA,ID,IS,JI,LA,NR,PA,PD,PG,PI,PT,PU,PY,RP,SC,SN,SO,TC,TI,UT,VL,WC,DI,PM,RI,EI,OI,CL,CT,CY,SP,SU,BE,BN,HO,PN,SE,MA,SI,GP,CA,BA,BF
305,We investigate the neutrino masses and mixing pattern in a version of the SU(3)(c)circle times SU(3)(L)circle times U(1)(X) model with one extra exotic charged lepton per family as introduced by O...,"Ponce, William A.\nZapata, Oscar\n",093007\n,"Ponce, WA\nZapata, O\n",1095\n,embryo implantation; trophoblast; cytokines; prostaglandins; aspirin\n,Article\n,cpossa@udea.edu.co\n,1100\n,Centro de Investigaciones of the Universidad de Antioquia\n,"We thank the Centro de Investigaciones of the Universidad de Antioquia\nfor financing this Study, Brian C. Bock for his comments and help with\nthe English, Juan Manuel Daza and Carlos Gutierrez f...",110VP\n,MAJORANA NEUTRINO MASSES; RIGHT-HANDED NEUTRINOS; ELECTROWEAK\nINTERACTIONS; CP VIOLATION; MODEL; MECHANISM; NUMBER;\nSU(3)(L)CIRCLE-TIMES-U(1)(N); QUARKS\n,9\n,Phys. Rev. D\n,English\n,47,"ONE PHYSICS ELLIPSE, COLLEGE PK, MD 20740-3844 USA\n",NOV\n,7,COLLEGE PK\n,J\n,AMERICAN PHYSICAL SOC\n,2006,"Ponce, WA (reprint author), Univ Antioquia, Inst Fis, Apartado Aereo 1226, Medellin, Colombia.\n",Astronomy & Astrophysics; Physics\n,1550-7998\n,PHYSICAL REVIEW D\n,8,Lepton masses and mixing without Yukawa hierarchies\n,WOS:000242409700018\n,74\n,"Astronomy & Astrophysics; Physics, Particles & Fields\n",10.1103/PhysRevD.74.093007\n,17122924.0,"Tschiptschin, Andre/F-7392-2012\n",1528-0020\n,"Carvajal-Carmona, Luis/0000-0001-7129-2918; \n","LOS ANGELES, CA\n",1985 ANNUAL CONVENTION OF THE AMERICAN PSYCHOLOGICAL ASSOC\n,1985\n,AMER PSYCHOL ASSOC\n,2\n,"Alarcon, R\nCole, PL\nDjalali, C\nUmeres, F\n",978-0-7354-0461-8\n,McMaster Univ\n,1\n,AIP CONFERENCE PROCEEDINGS\n,446\n,SI\n,NaN,NaN,NaN,NaN
996,"The parasitic infection neurocysticercosis may give rise to a variety of psychiatric manifestations that resemble, but are different from, primary psychiatric disorders. The aim of this study was ...","Sanzon, F\nOsorio, AM\nMorales, JP\nIsaza, R\nCardona, E\nMoncayo, LC\nVillota, GE\nZapata, OT\nPalacio, CA\nArbelaez, MP\nRestrepo, BI\n",PII S0165-5728(02)00101-7\n,"Sanzon, F\nOsorio, AM\nMorales, JP\nIsaza, R\nCardona, E\nMoncayo, LC\nVillota, GE\nZapata, OT\nPalacio, CA\nArbelaez, MP\nRestrepo, BI\n",532\n,neurocysticercosis; psychosis; seizures; serology; mental alteration;\neducation level; psychiatric\n,Article\n,germain.puzo@ipbs.fr\n,538\n,"CSICCONICYT from Uruguay; PEDECIBA from Uruguay; INCO-DC Programme of\nthe Commission of the European Communities (DG XII: Science, Research\nand Development), Brussels, EU [IC18-CT98-0366]; AVINA...","This work was partially supported by CSICCONICYT, and PEDECIBA from\nUruguay. Moreover, it benefited from international collaboration through\nthe ECLAT network. Additional financial support was o...",556GM\n,LINKED IMMUNOELECTROTRANSFER BLOT; TAENIA-SOLIUM; GLYCOPROTEIN ANTIGENS;\nNEUROCYSTICERCOSIS\n,6\n,Trop. Med. Int. Health\n,English\n,20,"9600 GARSINGTON RD, OXFORD OX4 2DG, OXON, ENGLAND\n",JUN\n,7,OXFORD\n,J\n,BLACKWELL PUBLISHING LTD\n,2002,"Restrepo, BI (reprint author), Univ Texas, UT Houston, Sch Publ Hlth, Brownsville Campus,80 Ft Brown SPH,2nd Floor,Rm 2, Brownsville, TX 78520 USA.\n","Public, Environmental & Occupational Health; Tropical Medicine\n",1360-2276\n,TROPICAL MEDICINE & INTERNATIONAL HEALTH\n,4,Serological screening for cysticercosis in mentally altered individuals\n,WOS:000175841100009\n,7\n,"Public, Environmental & Occupational Health; Tropical Medicine\n",10.1046/j.1365-3156.2002.00886.x\n,12031076.0,"Anaya, Juan-Manuel/J-1960-2016; \n",1567-7257\n,"Anaya, Juan-Manuel/0000-0002-6444-1249; Universidad del Rosario,\nBiblioteca/0000-0003-3491-9392; Correa, Paula/0000-0002-0941-9700\n",JAPAN\n,5th Annual Meeting of the Japan-Endocrine-Pathology-Society\n,"SEP, 2001\n",Japan Endocrine Pathol Soc\n,S\n,"Righini, GC\nConsortini, A\n",0-8194-4596-7\n,UNIV CALIF DAVIS\n,2\

In [19]:
'a\n'.strip()

'a'

In [15]:
kk=pd.DataFrame()
if kk.shape[0]==0:
    print(1)

1


[Develompent notebook](./dev.ipynb)